In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from bids.modeling import BIDSStatsModelsGraph
from bids.layout import BIDSLayout
from bids.tests import get_test_data_path
from os.path import join
import json

In [2]:
layout_path = join(get_test_data_path(), 'ds005')
layout = BIDSLayout(layout_path)
json_file = join(layout_path, 'models', 'ds-005_type-test_model.json')
spec = json.load(open(json_file, 'r'))

In [3]:
graph = BIDSStatsModelsGraph(layout, spec) #initialize graph

c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\site-packages\bids\modeling\statsmodels.py:63: UserWarning: [Node run]:Transformations reformatted to {'transformer': 'pybids-transforms-v1', 'instructions': [{'name': 'Factor', 'input': 'trial_type'}, {'name': 'Rename', 'input': 'trial_type.parametric gain', 'output': 'gain'}]}
  warnings.warn(f"[Node {node['name']}]:"


# Load variables from BIDS dataset¶

We will typically want to load variables into BIDS “collections” from the BIDS project. The `scan_length` argument is only necessary here because the test dataset doesn’t actually include nifti image headers, so duration of scans can’t be inferred. In typical use, you can call this without arguments.

In [4]:
graph.load_collections(scan_length=480)

c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\site-packages\bids\variables\io.py:276: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  _data = _data.apply(pd.to_numeric, errors='ignore')
c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\site-packages\bids\variables\io.py:276: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  _data = _data.apply(pd.to_numeric, errors='ignore')
c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\site-packages\bids\variables\io.py:276: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  _data = _data.apply(pd.to_numeric, errors='ignore')
c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\sit

# Access specific nodes¶

Now that the graph is loaded, we can start interacting with its nodes. We’ll typically start with the root node, which will usually contain the run-level model information. We can either access the `.root_node`, or use `get_node()` to get the node by its unique name (defined in the JSON file).

In [5]:
# Equivalent to calling graph.get_node('run') in this case.
root_node = graph.root_node

In [6]:
colls = root_node.get_collections()
first_sub = colls[0] # Collection for the first subject / run 
first_sub.to_df(entities=False) ### inicia um grafo aleatório

c:\Users\angel\Documents\pilotStudy_INPD\fmrianalysis\Lib\site-packages\bids\variables\collections.py:181: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.reset_index().replace("n/a", fillna)


,duration,onset,PTval,RT,distance from indifference,gain,loss,parametric gain,parametric loss,respcat,respnum,trial_type
0,3,0,5.15,0.000,NaN,20,15,-0.139,NaN,-1,0,parametric gain
1,3,4,6.12,1.793,NaN,18,12,-0.189,NaN,1,2,parametric gain
2,3,8,-4.85,1.637,NaN,10,15,-0.389,NaN,0,3,parametric gain
3,3,18,18.16,1.316,NaN,34,16,0.211,NaN,1,1,parametric gain
4,3,24,13.05,1.670,NaN,18,5,-0.189,NaN,1,1,parametric gain
...,...,...,...,...,...,...,...,...,...,...,...,...
81,3,458,-1.82,1.785,NaN,16,18,-0.239,NaN,0,4,parametric gain
82,3,462,24.12,1.280,NaN,36,12,0.261,NaN,1,2,parametric gain
83,3,466,17.09,1.394,NaN,26,9,0.011,NaN,1,1,parametric gain
84,3,470,8.12,1.249,NaN,20,12,-0.139,NaN,1,2,parametric gain


In [7]:
first_sub.variables
#tipo de variável presente no dataset

{'respcat': <SparseRunVariable(name='respcat', source='events')>,
 'gain': <SparseRunVariable(name='gain', source='events')>,
 'PTval': <SparseRunVariable(name='PTval', source='events')>,
 'trial_type': <SparseRunVariable(name='trial_type', source='events')>,
 'parametric loss': <SparseRunVariable(name='parametric loss', source='events')>,
 'loss': <SparseRunVariable(name='loss', source='events')>,
 'parametric gain': <SparseRunVariable(name='parametric gain', source='events')>,
 'respnum': <SparseRunVariable(name='respnum', source='events')>,
 'distance from indifference': <SparseRunVariable(name='distance from indifference', source='events')>,
 'RT': <SparseRunVariable(name='RT', source='events')>}

In [9]:
# force_dense controls whether the output for run-level design matrices
# will be resampled to a uniform "dense" representation, or left alone
# as a sparse representation if possible.
outputs = root_node.run(group_by=['run', 'subject'], force_dense=False, transformation_history=True)
len(outputs)
outputs[:10]    

[<BIDSStatsModelsNodeOutput(name=run, entities={'run': 1, 'subject': '01'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 2, 'subject': '01'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 3, 'subject': '01'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 1, 'subject': '02'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 2, 'subject': '02'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 3, 'subject': '02'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 1, 'subject': '03'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 2, 'subject': '03'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 3, 'subject': '03'})>,
 <BIDSStatsModelsNodeOutput(name=run, entities={'run': 1, 'subject': '04'})>]

In [10]:
outputs[0].X

,RT,gain,RT:gain
0,0.000,1.0,0.000
1,1.793,1.0,1.793
2,1.637,1.0,1.637
3,1.316,1.0,1.316
4,1.670,1.0,1.670
...,...,...,...
81,1.785,1.0,1.785
82,1.280,1.0,1.280
83,1.394,1.0,1.394
84,1.249,1.0,1.249
